## Get dataset

In [7]:
import os
import gspread
import pandas as pd

json_path = os.path.join(os.getcwd(), '../', 'data', 'raw', 'service_account.json')

sa = gspread.service_account(filename=json_path)
sh = sa.open('ProyectoDM')

worksheet = sh.sheet1
data = worksheet.get_all_records()
df = pd.DataFrame(data)

## Main

In [8]:
renamed_columns = {
    # 'Marca temporal': 'marca_temporal',
    # 'Dirección de correo electrónico': 'correo',
    '¿A qué escuela perteneces?': 'escuela',
    '¿Cuál es tu promedio global? ': 'promedio_global',
    '¿Cuántas materias has reprobado a lo largo de tu trayectoria?': 'materias_reprobadas',
    '¿Qué edad tienes? ': 'edad',
    '¿Cuál es tu género?': 'genero',
    '¿Tienes dependientes económicos?': 'dependientes_economicos',
    '¿Tienes personas a tu cuidado?': 'personas_a_cargo',
    '¿Te sientes apoyado por tu familia?': 'apoyo_familiar',
    '¿Requieres de apoyos económicos (becas) para sostener tus estudios?': 'apoyos_economicos',
    '¿Eres foráneo?': 'foraneo',
    '¿Cuánto tiempo tardas en llegar a tu escuela en minutos?': 'minutos_traslado',
    '¿Cuántas horas diarias dedicas al estudio fuera de clases?': 'horas_estudio',
    'Para validar la lectura de esta encuesta selecciona la opción 3 horas': 'validacion_encuesta',
    '¿Cuántas horas trabajas al día? \nSi no trabajas, escribe \'0\'.': 'horas_trabajo',
    '¿Padeces de alguna enfermedad crónica (Por ejemplo: diabetes, hipertensión, asma, etc.)?': 'enfermedad_cronica',
    '¿Te gusta tu carrera?': 'satisfaccion_carrera',
    '¿Piensas ejercer la carrera?': 'ejercer_carrera',
    '¿Cuántas horas duermes en promedio al día?': 'horas_sueno',
    '¿Con qué frecuencia utilizas herramientas de IA?': 'frecuencia_IA',
}

df.drop(columns=['Marca temporal', 'Dirección de correo electrónico'], inplace=True)
df.rename(columns=renamed_columns, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1833 entries, 0 to 1832
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   escuela                  1833 non-null   object 
 1   promedio_global          1833 non-null   float64
 2   materias_reprobadas      1833 non-null   float64
 3   edad                     1833 non-null   int64  
 4   genero                   1833 non-null   object 
 5   dependientes_economicos  1833 non-null   object 
 6   personas_a_cargo         1833 non-null   object 
 7   apoyo_familiar           1833 non-null   int64  
 8   apoyos_economicos        1833 non-null   object 
 9   foraneo                  1833 non-null   object 
 10  minutos_traslado         1833 non-null   float64
 11  horas_estudio            1833 non-null   float64
 12  validacion_encuesta      1833 non-null   object 
 13  horas_trabajo            1833 non-null   float64
 14  enfermedad_cronica      

In [11]:
# Filter rows with value 3 in validacion_encuesta column and count them
valid_rows = df[df['validacion_encuesta'] == '3 horas']
print(f'[+] Valid rows: {len(valid_rows)}')

# Filter rows with value is not 3 in validacion_encuesta column and count them
invalid_rows = df[df['validacion_encuesta'] != '3 horas']
print(f'[+] Invalid rows: {len(invalid_rows)}')


[+] Valid rows: 1777
[+] Invalid rows: 56


In [12]:
from sklearn.preprocessing import LabelEncoder

# Store mappings
mappings = {}

columns_list = [
    'escuela', 'genero', 'dependientes_economicos', 'personas_a_cargo',
    'apoyos_economicos', 'foraneo', 'validacion_encuesta',
    'enfermedad_cronica', 'satisfaccion_carrera', 'ejercer_carrera'
]

for column in columns_list:
    label_encoder = LabelEncoder()
    
    # Fit and transform the column
    df[f'{column}_encoded'] = label_encoder.fit_transform(df[column])
    
    # Store the mapping of old values to new values
    mappings[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Display the old and new values
for column, mapping in mappings.items():
    print(f"Column: {column}")
    for original, encoded in mapping.items():
        print(f"  {original} -> {encoded}")
    print("\n")


Column: escuela
  ENCB -> 0
  ESCOM -> 1
  ESFM -> 2
  ESIA -> 3
  ESIME -> 4
  ESIQIE -> 5
  ESIT -> 6
  EST -> 7
  UPIEM -> 8


Column: genero
  Femenino -> 0
  Masculino -> 1
  Prefiero no decirlo -> 2


Column: dependientes_economicos
  No -> 0
  Sí -> 1


Column: personas_a_cargo
   -> 0
  No -> 1
  Sí -> 2


Column: apoyos_economicos
  No -> 0
  Sí -> 1


Column: foraneo
  No -> 0
  Sí -> 1


Column: validacion_encuesta
  1 hora -> 0
  2 horas -> 1
  3 horas -> 2


Column: enfermedad_cronica
  No -> 0
  Sí -> 1


Column: satisfaccion_carrera
  Indiferente -> 0
  No -> 1
  Sí -> 2


Column: ejercer_carrera
  No -> 0
  No estoy seguro -> 1
  Sí -> 2




## Save dataset processed

In [13]:
# Remove columns that have not been encoded
df.drop(columns=[
    'escuela', 'genero', 'dependientes_economicos', 'personas_a_cargo',
    'apoyos_economicos', 'foraneo', 'validacion_encuesta',
    'enfermedad_cronica', 'satisfaccion_carrera', 'ejercer_carrera'
], inplace=True)

# Rename the encoded columns
df.rename(columns={
    'escuela_encoded': 'escuela',
    'genero_encoded': 'genero',
    'dependientes_economicos_encoded': 'dependientes_economicos',
    'personas_a_cargo_encoded': 'personas_a_cargo',
    'apoyos_economicos_encoded': 'apoyos_economicos',
    'foraneo_encoded': 'foraneo',
    'validacion_encuesta_encoded': 'validacion_encuesta',
    'enfermedad_cronica_encoded': 'enfermedad_cronica',
    'satisfaccion_carrera_encoded': 'satisfaccion_carrera',
    'ejercer_carrera_encoded': 'ejercer_carrera'
}, inplace=True)

# Save the processed data
df.to_csv('../data/processed/processed_data.csv', index=False)